In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("netflix.sqlite")
cur = con.cursor()
df = pd.read_sql("select * from netflix_titles", sqlite3.connect("netflix.sqlite"))

In [3]:
print(df['show_id'].shape[0])
print(len(df['show_id'].unique()))
df.sample(3)

6234
6234


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
2728,80158580,Movie,Amelia: A Tale of Two Sisters,Edward Cotterill,Rachael Stirling,United Kingdom,"June 1, 2017",2017,TV-PG,44 min,Documentaries,"Eight decades after her disappearance, Amelia ..."
4693,70236586,Movie,Tezz,Priyadarshan,"Anil Kapoor, Ajay Devgn, Boman Irani, Kangna R...","United Kingdom, India","December 1, 2017",2012,TV-MA,121 min,"Action & Adventure, International Movies, Musi...",This high-octane thriller tells the story of a...
664,80177406,Movie,Patton Oswalt: Annihilation,Bobcat Goldthwait,Patton Oswalt,United States,"October 17, 2017",2017,TV-MA,66 min,Stand-Up Comedy,"By turns scathing and candid, Patton Oswalt re..."


In [4]:
actors_df = df['cast'].str.split(', ')

In [5]:
actors = set()
for line in actors_df:
    for actor in line:
        t = actor.strip()
        if t != '':
            actors.add(t)
len(actors)

27405

In [6]:
with con: 
    cur.execute("""
        CREATE TABLE IF NOT EXISTS actor (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name TEXT
        );
    """)

In [7]:
with con:
    for actor in actors:
        cur.execute("INSERT INTO actor (name) values(?)", (actor,))

In [8]:
with con: 
    cur.execute("""
        CREATE TABLE IF NOT EXISTS actor_show (
            actor TEXT,
            show_id INT
        );
    """)

In [9]:
%%time
count_last_upd = len(df)
for i in range(len(df) - count_last_upd, len(df)):
    for actor in actors_df.iloc[i]:
        if not actor:
            continue
        with con:
            cur.execute("INSERT INTO actor_show (actor, show_id) values(?, ?)", (actor, int(df.iloc[i]['show_id'])))

CPU times: total: 1min 13s
Wall time: 4min 52s


In [10]:
df_as = pd.read_sql("select * from actor_show", sqlite3.connect("netflix.sqlite"))
df_as

,actor,show_id
0,Alan Marriott,81145628
1,Andrew Toth,81145628
2,Brian Dobson,81145628
3,Cole Howard,81145628
4,Jennifer Cameron,81145628
...,...,...
44306,Courteney Cox,70153404
44307,Lisa Kudrow,70153404
44308,Matt LeBlanc,70153404
44309,Matthew Perry,70153404


In [11]:
with con: 
    cur.execute("""
        SELECT t1.actor as value_1,
               t2.actor as value_2,
               count(t1.show_id)
        FROM actor_show t1
        CROSS JOIN
               actor_show t2
        where t1.show_id = t2.show_id and t1.actor != t2.actor
        group by t1.actor, t2.actor
        order by count(t1.show_id) DESC
        limit 1
    """)
cur.fetchone()[:-1]

('John Paul Tremblay', 'Robb Wells')

In [12]:
cur.close()